In [39]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_rows',500)

In [14]:
df = pd.read_csv(r"C:\Users\ankar.yadav\Downloads\P14-Artificial-Neural-Networks\Artificial_Neural_Networks\Churn_Modelling.csv")

In [15]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [16]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [41]:
predictors = ['CreditScore', 'Geography','Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
              'IsActiveMember', 'EstimatedSalary',]

qual_preds = ['Geography','Gender']

def classifier_model(df, target, predictors,qual_preds, dummy = True):
    result = pd.DataFrame()
    df_pre = df.copy()
    scaler = MinMaxScaler()
    
    if dummy == True:
        df_pre = pd.get_dummies(df_pre, columns = qual_preds, drop_first= True)
    
    y = df[target]
    X = df_pre
    scale = list(set(predictors) - set(qual_preds))
    X[scale] = scaler.fit_transform(X[scale])
        
    X_train = X.iloc[:-2000,:]
    X_test = X.iloc[-2000:,:]
    y_train = y.iloc[:-2000]
    y_test = y.iloc[-2000:]
    X_train = X_train.drop(['CustomerId','Surname'],axis =1)
    X_test = X_test.drop(['CustomerId','Surname'],axis =1)
    
    classifier = Sequential()
    classifier.add(Dense(units = 12, activation = 'relu', kernel_initializer = 'uniform', input_shape = (13,)))
    classifier.add(Dense(units = 12, activation = 'relu', kernel_initializer = 'uniform',))
    
    classifier.add(Dense(units = 1, activation = 'sigmoid', kernel_initializer = 'uniform',))
    
    classifier.compile(loss= 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    classifier.fit(X_train, y_train, batch_size = 10, epochs= 10 )
    
    y_pred = classifier.predict(X_test)
    y_pred_1 = [1 if i[0]>0.5 else 0 for i in y_pred]
    y_pred = pd.Series([i[0] for i in y_pred])
    cm = confusion_matrix(y_test,y_pred_1)
    
    
    return cm
    
    
    
    
    

In [42]:
model = classifier_model(df, 'Exited', predictors,qual_preds, dummy = True)

Epoch 1/10
8000/8000 [==============================] - 1s 101us/sample - loss: 0.5410 - accuracy: 0.7912
Epoch 2/10
8000/8000 [==============================] - 1s 87us/sample - loss: 0.5258 - accuracy: 0.7941
Epoch 3/10
8000/8000 [==============================] - 1s 92us/sample - loss: 0.5161 - accuracy: 0.7941
Epoch 4/10
8000/8000 [==============================] - 1s 91us/sample - loss: 0.5057 - accuracy: 0.7941
Epoch 5/10
8000/8000 [==============================] - 1s 87us/sample - loss: 0.5003 - accuracy: 0.7941
Epoch 6/10
8000/8000 [==============================] - 1s 87us/sample - loss: 0.4675 - accuracy: 0.7941
Epoch 7/10
8000/8000 [==============================] - 1s 87us/sample - loss: 0.3882 - accuracy: 0.8112
Epoch 8/10
8000/8000 [==============================] - 1s 89us/sample - loss: 0.2579 - accuracy: 0.9079
Epoch 9/10
8000/8000 [==============================] - 1s 87us/sample - loss: 0.1393 - accuracy: 0.9606
Epoch 10/10
8000/8000 [==============================]

In [43]:
cm = model

In [44]:
cm

array([[1589,   21],
       [   0,  390]], dtype=int64)